In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada
gpus= tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
tf.config.experimental.set_memory_growth(gpus[0], True) # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V2 import *
import aux_func_V3 as Modelo1
import aux_func_numpy as debug

# Importando módulo do Weights & Biases para fazer a varredura dos hiper-parâmetros.
import wandb
from wandb.keras import WandbCallback

from IPython.display import clear_output # Definindo a precisão padrão para as análises

In [4]:
# Lendo o conjunto de dados que será utilizado para o treino da rede neural

# Importando o pacote de manipulação de arquivos
import dill

# Lendo os dados
with open('data.bck','rb') as file:
    # Utilizando a CPU como processador
    with tf.device('/device:CPU:0'):
        data_full = dill.load(file)
        data_full = tf.transpose(data_full[0],[1,0,2]).numpy()

In [5]:
# Formatando conjunto de dados para o treino da rede neural
data_x = np.squeeze(data_full[:,:,1:])
data_y = np.squeeze(data_full[:,:,:-1])

data_x = data_x[:,np.arange(100)*4]
data_y = data_y[:,np.arange(100)*4]
data_x = np.concatenate(np.split(data_x, data_x.shape[2], axis=2), axis=0)
data_y = np.concatenate(np.split(data_y, data_y.shape[2], axis=2), axis=0)

indices = np.where(np.sum(data_x**2, axis=1) > 0)[0]
data_x  = data_x[indices]
data_y  = data_y[indices]

# Aleatorizando a ordem dos elementos dos dados
indice = np.arange(data_x.shape[0])
np.random.shuffle(indice)
data_x = data_x.astype('float64')[indice]
data_y = data_y.astype('float64')[indice]

In [6]:
test_size=5000

# Conjunto de dados de treino
train_x=data_x[:-test_size]
train_y=data_y[:-test_size]

# Conjunto de dados de teste
test_x=data_x[-test_size:]
test_y=data_y[-test_size:]

In [7]:
# Inicializando a configuração padrão de treino.
default_config = wandb.config

default_config.conv_size=4
default_config.regul_ativ=0 
default_config.regul_weigth=0.428
default_config.weigth_OF=0.9159
default_config.usa_reg_ativ=0
default_config.batch_size=256
default_config.deep=8
default_config.learning_rate=0.02645
default_config.Model='Modelo 1'
default_config.ativ_func='sigmoid'

# Definindo a configuração da varredura.
sweep_config = {
    'method': 'bayes', # Método de busca.
    'metric': {
      'name': 'val_Inf_Norm', # Métrica alvo.
      'goal': 'minimize'      # Objetivo.
    },
    'parameters': { # Parâmetros a serem modificados e a suas respectivas distribuições à priori.
        'conv_size': {
            'values': [2**i for i in range(5)]
        },
        'Model': {
            'values': ['Modelo 1', 'Modelo 2']
        },
        'regul_ativ': {
            'distribution': 'log_uniform',
            'min': -5.0,
            'max': -1.0
        },
        'usa_reg_ativ': {
            'values':[0.0,1.0]
        },
        'regul_weigth': {
            'distribution': 'log_uniform',
            'min': -5.0,
            'max': -1.0
        },
        'weigth_OF':{
            'distribution': 'uniform',
            'min': 0.0,
            'max': 1.0
        },
        'ativ_func':{
            'values':['sigmoid','elu']
        },
        'deep': {
            'values': [i for i in range(5)]
        },
        'batch_size': {
            'values': [2**i for i in range(6,8)]
        },
        'learning_rate': {
            'distribution': 'log_uniform',
            'min': -5.0,
            'max': -1.0
        }
    }
}

# Criando varredura.
sweep_id = wandb.sweep(sweep_config, project='WENO-DS', entity='silvaneojunior')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: avxz1gc0
Sweep URL: https://wandb.ai/silvaneojunior/WENO-DS/sweeps/avxz1gc0


In [8]:
def Inf_Norm(y_true, y_pred):
    """
    Função que recebe dois tensors contendo valores reais e retorna a Norma infinita da diferença entre eles.
    -------------------------------------------------------------------------------------------
    y_true       (tensor): valores de referência do conjunto de dados.
    y_pred       (float): valores de previstos pela rede.
    -------------------------------------------------------------------------------------------
    error        (tensor): Erro de previsão na Norma infinita.
    -------------------------------------------------------------------------------------------
    """
    y_true = tf.cast(y_true, y_pred.dtype)
    error=tf.reduce_max(tf.math.abs(y_pred - y_true), axis=-1)
    return error

def train():
    """
    Função que realiza o treino da rede neural para um conjunto de hiper-parâmetros
    -------------------------------------------------------------------------------------------
    Sem argumentos
    -------------------------------------------------------------------------------------------
    Sem output
    -------------------------------------------------------------------------------------------
    """
    clear_output() # Limpa o output da célula (exclusivo para o Jupyter)
    run=wandb.init() # Inicializando o treino da rede. Nesta etapa o wandb amostrará um conjunto de hipe-parâmetros segundo a distribuição à posterior.
    config=wandb.config # Armazenando a configuração amostrada.
    
    # Definindo callbacks.
    # Cada função da lista a seguir será executada ao final de cada epoch.
    callback = [WandbCallback(),tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)]
    
    if config.Model=='Modelo 1' or config.ativ_func=='sigmoid':
        # Configuração para a função de ativação sigmoid.
        ativ_func='sigmoid'
        regul_func=lambda x: -config.usa_reg_ativ*config.regul_ativ*tf.math.reduce_sum(tf.math.log(x))
    else:
        # Configuração para a função de ativação elu+1.
        ativ_func=lambda x: tf.nn.elu(x)+1
        regul_func=lambda x: config.usa_reg_ativ*config.regul_ativ*tf.math.reduce_sum((x-1)**2)

    class Burguers_layer(keras.layers.Layer):
        """Criando uma camada de rede neural cuja superclasse é a camada
        do keras para integrar o algoritmo do WENO com a rede neural"""
        def __init__(self, t_final, Δx, CFL, fronteira):
            """
            Construtor da classe
            --------------------------------------------------------------------------------------
            t_final      (float): tamanho máximo da variação temporal
            Δx           (float): distância espacial dos pontos na malha utilizada
            CFL          (float): constante utilizada para determinar o tamanho da malha temporal
            fronteira (function): função que determina o comportamento do algoritmo na fronteira
            --------------------------------------------------------------------------------------
            """
            super(Burguers_layer, self).__init__(dtype='float64') # Chamando o inicializador da superclasse
            
            # Salvando os valores dos parâmetros do construtor no objeto
            self.t_final = tf.cast(t_final,'float64')
            self.Δx = Δx
            self.CFL=CFL
            self.fronteira = fronteira
        def build(self, input_shape):
            """
            Função para compor as camadas que constituem essa camada da rede neural
            ------------------------------------------------------------------------
            input_shape : não é utilizado por essa função, mas é um argumento obrigatório para camadas do Keras.
            ------------------------------------------------------------------------
            """
            self.layers=[]
            wei_reg=tf.keras.regularizers.L2(config.regul_weigth)
            for i in range(config.deep):
                self.layers.append(tf.keras.layers.ZeroPadding1D(padding=2))
                self.layers.append(keras.layers.Conv1D(config.conv_size,5,activation='elu',dtype=data_x.dtype,kernel_regularizer=wei_reg))
            if config.Model!='Modelo 1':
                self.layers.append(keras.layers.Conv1D(1,1,activation=ativ_func,dtype=data_x.dtype,kernel_regularizer=wei_reg,activity_regularizer=regul_func))
            else:
                self.layers.append(keras.layers.Conv1D(1,1,activation=ativ_func,dtype=data_x.dtype,kernel_regularizer=wei_reg,activity_regularizer=regul_func))
            
        def network_graph(self,x):
            """
            Função utilizado para executar sucessivamente as camadas dessa camada 
            da rede neural, passando o input de uma para a próxima
            ----------------------------------------------------------------------
            x (tensor): valor de entrada da rede
            ----------------------------------------------------------------------
            y (tensor): valor de saída da rede
            ----------------------------------------------------------------------
            """
            y=x
            for layer in self.layers:
                y=layer(y)
            return y

        def call(self, inpt,mask=None):
            """
            Função utilizada para treino que recebe o input da camada e computa o output
            -----------------------------------------------------------------------------
            inpt  (tensor): input passado para a camada
            mask       (?): não é utilizado por essa função
            -----------------------------------------------------------------------------
            outpt (tensor): output retorna pela camada
            -----------------------------------------------------------------------------
            """
            Λ  = tf.math.reduce_max(tf.abs(inpt)) # Valor utilizado para obter o Δt
            Δt = self.Δx*self.CFL/Λ               # Obtendo o valor de Δt a partir de CFL
            # Caso o passo temporal utrapasse o valor de t_final
            # então o tamanho do passo se torna o tempo que 
            # falta para se obter o t_final
            Δt = tf.where(Δt > self.t_final, self.t_final, Δt)
            
            if config.Model!='Modelo 1':
                # Grafo do WENO-Z para o Modelo 2
                return Graph_Burgers(u=inpt, Δt=Δt, Δx=self.Δx, fronteira=self.fronteira,network=self.network_graph)
            else:
                # Grafo do WENO-Z para o Modelo 1
                return Modelo1.Graph_Burgers(u=inpt, Δt=Δt, Δx=self.Δx, fronteira=self.fronteira,network=self.network_graph)
        def predict(self, inputs,mask=None):
            """Função que faz previsão a partir de um input"""
            return self(inputs,mask=None) # Chamando o função call (self.predict(inputs, mask) = self.call(inpt, mask))

    # Definindo o input da rede e o otimizador de treino
    input_x=keras.layers.Input([100,1],dtype='float64')
    optimizer = keras.optimizers.Adam(learning_rate=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
    
    # Criando uma camada de Burgers que integra o WENO à rede neural
    final_layer=Burguers_layer(t_final=0.01,Δx=0.04,CFL=0.5,fronteira=FronteiraPeriodica)

    class MES_OF(tf.keras.losses.Loss):
        """Criando uma função de custo cuja superclasse é a de funções de
        custo do keras"""
        def call(self, y_true, y_pred):
            """
            Função que avalia o custo dado um valor de referência e um valor previsto
            --------------------------------------------------------------------------
            y_true (tensor): valor de referência
            y_pred (tensor): valor predito
            --------------------------------------------------------------------------
            loss   (tensor): custo associado
            --------------------------------------------------------------------------
            """
            y_true = tf.cast(y_true, y_pred.dtype) # Convertendo os tipos para evitar conflitos
            y_min  = tf.math.reduce_min(y_true,axis=1,keepdims=True)
            y_max  = tf.math.reduce_max(y_true,axis=1,keepdims=True)

            loss = (1-config.weigth_OF)*tf.reduce_mean(
                tf.math.square(y_pred - y_true), axis=-1) + \        # Custo por erro na norma L2
                config.weigth_OF*tf.reduce_sum(
                    tf.where(y_pred > y_max, y_pred - y_max,  0) + \ # Custo por previsão 'muito alta'
                    tf.where(y_pred < y_min, y_min  - y_pred, 0),    # Custo por previsão 'muito baixa'
                axis=-1)
            return loss
        
    # Criando a rede neural
    Network=keras.Model(input_x, final_layer(input_x))
    # Configurando a função de perda e o otimizador
    Network.compile(loss=MES_OF(),optimizer=optimizer,metrics=[Inf_Norm,'mean_absolute_error'])
    
    # Treinando a rede neural
    loss=Network.fit(train_x                                        , # Dados de treino
                     train_y                                        , # Dados de treino
                     validation_data = (test_x, test_y)             , # Dados de validação
                     batch_size      = config.batch_size            , # Tamanho do batch
                     epochs          = int(3/config.learning_rate)  , # Número de epochs
                     steps_per_epoch = 30                           , # Número de batchs por epoch
                     shuffle         = True                         , # Aleatorização dos batchs
                     callbacks=callback                             ) # Funções a serem executadas no final de cada Epoch
    
    # Calculando os erros de previsão utilizando o WENO-Z em uma malha mais fina 
    # como solução de referência e depois calculando o WENO-Z e o WENO-Z com a 
    # modificação da rede neural numa malha mais grossa

    Δx = 0.04                                  # Distância espacial dos pontos na malha mais grossa utilizada
    x  = tf.range(-2, 2, Δx, dtype=float_pres) # Gerando a malha de pontos no espaço unidimensional

    full_U   = test_x[-500:] # 500 últimos dados de entrada do conjunto de treino
    ref_full = test_y[-500:] # 500 últimos dados de saída do conjunto de treino

    CFL    = 0.5  # Constante utilizada para determinar o tamanho da malha temporal
    Δt_max = 0.01 # Instante de tempo final
    t      = 0.0  # Instante de tempo inicial

    # Valor utilizado para obter o Δt
    Λ = tf.math.reduce_max(tf.abs(full_U), axis=1, keepdims=True)
    # Obtendo o valor de Δt a partir de CFL
    Δt = Δx*CFL/Λ
    # Caso o passo temporal utrapasse o valor de Δt_max então o 
    # tamanho do passo se torna Δt_max
    Δt = tf.where(Δt > Δt_max, Δt_max, Δt)

    net_u   = tf.squeeze(Network.predict(full_U))             # Previsão com o WENO-Z modificado pela rede neural
    debug_u = Burgers(full_U,Δt, Δx, CFL, FronteiraPeriodica) # Previsão com o WENO-Z

    # Armazenando ambos os erros de previsão
    error = tf.stack([tf.squeeze(net_u)-tf.squeeze(ref_full),tf.squeeze(tf.squeeze(debug_u)-tf.squeeze(ref_full))],axis=0)
    
    # Erro médio na norma L2
    desv_error_MSE=tf.math.reduce_mean((tf.math.reduce_sum(error**2,axis=1)**0.5)/error.shape[1],axis=1)
    # Erro médio na norma do infinito
    desv_error_Inf=tf.math.reduce_max(tf.math.reduce_max(tf.math.abs(error),axis=1),axis=1)
    
    #Passando os erros para o Weigths and Biases.
    wandb.log({"Depara_MSE": (desv_error_MSE[0]/desv_error_MSE[1]),
               "Depara_Inf": (desv_error_Inf[0]/desv_error_Inf[1])})

In [ ]:
# Executando a varredura.
wandb.agent(sweep_id, train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/212
30/30 [==============================] - 25s 585ms/step - loss: 2.7784 - Inf_Norm: 0.0028 - mean_absolute_error: 0.0028 - val_loss: 0.4361 - val_Inf_Norm: 0.0033 - val_mean_absolute_error: 0.0033


wandb: ERROR Can't save model, h5py returned error: Layer Burguers_layer has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/212
30/30 [==============================] - 16s 536ms/step - loss: 0.2974 - Inf_Norm: 0.0031 - mean_absolute_error: 0.0031 - val_loss: 0.2238 - val_Inf_Norm: 0.0033 - val_mean_absolute_error: 0.0033
Epoch 3/212
30/30 [==============================] - 17s 558ms/step - loss: 0.1951 - Inf_Norm: 0.0028 - mean_absolute_error: 0.0028 - val_loss: 0.1745 - val_Inf_Norm: 0.0033 - val_mean_absolute_error: 0.0033
Epoch 4/212
30/30 [==============================] - 16s 539ms/step - loss: 0.1624 - Inf_Norm: 0.0031 - mean_absolute_error: 0.0031 - val_loss: 0.1515 - val_Inf_Norm: 0.0033 - val_mean_absolute_error: 0.0033
Epoch 5/212
30/30 [==============================] - 16s 545ms/step - loss: 0.1436 - Inf_Norm: 0.0030 - mean_absolute_error: 0.0030 - val_loss: 0.1362 - val_Inf_Norm: 0.0033 - val_mean_absolute_error: 0.0033
Epoch 6/212
30/30 [==============================] - 16s 537ms/step - loss: 0.1306 - Inf_Norm: 0.0032 - mean_absolute_error: 0.0032 - val_loss: 0.1250 - val_Inf_Norm: 0